# Exemplo 12: Agricultura - Irrigação Inteligente (Dask)

Este notebook demonstra o processamento de dados de sensores de solo para otimizar a agricultura de precisão usando **Dask**.

**Cenário**: Analisar umidade do solo em diferentes zonas de plantio e determinar a necessidade de irrigação.

## 1. Configuração

In [ ]:
!pip install -q dask[complete] distributed

In [ ]:
from dask.distributed import Client
client = Client()
client

## 2. Dados de Sensores de Solo

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os

os.makedirs('agri_data', exist_ok=True)

def generate_soil_data(zone_id):
    readings = 1000
    sensors = [f'SENS-{zone_id}-{i}' for i in range(1, 21)]
    
    df = pd.DataFrame({
        'sensor_id': np.random.choice(sensors, readings),
        'zone_id': f'ZONE-{zone_id}',
        'timestamp': pd.date_range(start='2024-01-01', periods=readings, freq='h'),
        'moisture_percent': np.random.uniform(10, 60, readings),
        'temperature_c': np.random.uniform(20, 35, readings)
    })
    
    # Simular seca na Zona 3
    if zone_id == 3:
        df['moisture_percent'] = df['moisture_percent'] * 0.5
        
    df.to_csv(f'agri_data/zone_{zone_id}.csv', index=False)

print("Gerando dados de 10 Zonas...")
for i in range(1, 11):
    generate_soil_data(i)

## 3. Decisão de Irrigação

In [ ]:
ddf = dd.read_csv('agri_data/zone_*.csv')

# Calcular umidade média por zona
zone_stats = ddf.groupby('zone_id')['moisture_percent'].mean()

# Se umidade < 30%, irrigar
def irrigation_need(moisture):
    if moisture < 30:
        return "ATIVAR"
    return "AGUARDAR"

# Dask computando
avg_moisture = zone_stats.compute()
decisions = avg_moisture.apply(irrigation_need)

print("Status da Irrigação:")
decision_df = pd.DataFrame({'Umidade Média': avg_moisture, 'Ação': decisions})
print(decision_df)

In [ ]:
client.close()